# Split Data into Training and Test

In [1]:
!pip install imblearn
!pip install xgboost
!pip install hyperopt

In [13]:
# load dependencies
import pandas as pd # for importing and handling data
import numpy as np # for working with arrays
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score, KFold # for splitting data and hyperparameter tuning
from sklearn.preprocessing import LabelEncoder # for processing data for xgboost
from imblearn.over_sampling import SMOTE # for smote oversampling
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials # for Bayesian hyperparameter tuning
import xgboost as xgb # for xgboost model

In [3]:
# load data
df = pd.read_csv("/home/jupyter/final_project/pisa_median.csv")
df.head()

,Unnamed: 0,ST352Q06JA,AGE,ST004D01T,DURECEC,REPEAT,MISSSC,SKIPPING,TARDYSD,EXPECEDU,...,ST353Q07JA,ST353Q08JA,ST348Q01JA,ST348Q02JA,ST348Q03JA,ST348Q04JA,ST348Q05JA,ST348Q06JA,ST348Q07JA,ST348Q08JA
0,1.0,2.0,15.58,1.0,2.0,0.0,0.0,1.0,2.0,7.0,...,2.0,2.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,2.0
1,2.0,2.0,16.17,2.0,2.0,0.0,1.0,0.0,1.0,7.0,...,4.0,4.0,2.0,3.0,2.0,3.0,4.0,4.0,4.0,4.0
2,3.0,4.0,15.58,2.0,0.0,0.0,0.0,0.0,0.0,9.0,...,2.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,2.0
3,4.0,4.0,15.42,2.0,2.0,0.0,0.0,0.0,0.0,4.0,...,1.0,2.0,1.0,3.0,4.0,1.0,4.0,1.0,2.0,1.0
4,5.0,2.0,15.75,2.0,1.0,0.0,0.0,0.0,0.0,8.0,...,2.0,2.0,1.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0


In [4]:
# split into target and features
y = df["ST352Q06JA"].values
X = df.drop(["ST352Q06JA", "Unnamed: 0"], axis=1).values

In [5]:
# split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, # common to use 20-30% of data as test set
                                                    random_state = 2001, # set seed equivalent
                                                    stratify = y) # churn amounts equal in train and test

In [6]:
print(X_train[0:3])
print(y_train[0:3])

[[ 1.5670e+01  2.0000e+00  2.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  8.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   6.0000e+00  1.0000e+01  0.0000e+00  6.0000e+00 -7.1900e-02 -1.2280e+00
   1.1246e+00  1.4750e-01 -7.1000e-03 -3.2030e-01  2.7474e+00 -9.7550e-01
   1.8227e+00  5.2030e-01  2.3650e+00  3.8830e+00  1.1187e+00  4.9584e+00
   1.3749e+00  2.4100e-02  1.3679e+00  3.1637e+00  1.7174e+00  1.7465e+00
   1.0481e+00  1.0000e+00  1.0000e+00  1.0000e+00  2.0000e+00  1.0000e+00
   2.0000e+00  1.0000e+00  3.0000e+00  3.0000e+00  4.0000e+00  3.0000e+00
   4.0000e+00  2.0000e+00  2.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00
   2.0000e+00  2.0000e+00  2.0000e+00  2.0000e+00  3.0000e+00  3.0000e+00
   3.0000e+00  4.0000e+00  4.0000e+00  2.0000e+00  1.0000e+00]
 [ 1.5830e+01  1.0000e+00  4.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00
   0.0000e+00  6.0000e+00  1.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  1.0000e+01  6.0000e+00  1.0000e+01

# Conduct Oversampling with SMOTE on Training Data

In [6]:
# frequency of observations in training data pre-oversampling
temp = y_train.astype(int)
np.bincount(temp)

array([   0, 1349, 2325, 2088, 3182])

In [7]:
# conduct oversampling on training data using SMOTE
# SMOTE overview here: https://www.youtube.com/watch?v=1Ic7GRtDrPM
smote = SMOTE(random_state = 2001)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [8]:
# convert y_smote to integer (needed for xgboost later)
y_smote = y_smote.astype(int)
np.bincount(y_smote)

array([   0, 3182, 3182, 3182, 3182])

In [9]:
# transform data to work with xgboost (encoding needs to start at zero)
le = LabelEncoder()
y_smote = le.fit_transform(y_smote)

# Hyperparameter Tuning and Cross-Validation for xgboost

We want to train hyperparameters for `eta` (learning rate), `n_estimators` (number of estimators), and `max_features` (maximum features considered at each split). Specifically, using the following values:

* `eta`: 0.1, 0.01, 0.001
* `n_estimators`: 50, 100, 250, 500
* `colsample_bytree`: 0.1, 0.5, 0.7, 1.0

Furthermore, we will conduct 5-fold cross-validation for each training model to return an aggregate RMSE value. In total, that yields 3 (`eta`) x 4 (`n_estimators`) x 4 (`colsample_bytree`) x 5 (cross-validation) = 240 estimated models. We will use randomized search for this endeavour.

In [18]:
# # EXAMPLE CODE, TO BE DELETED

# from sklearn.metrics import confusion_matrix, mean_squared_error as MSE

# # test single xgboost
# bst = xgb.XGBClassifier(eta=0.1, n_estimators=50, colsample_bytree=0.5)
# bst.fit(X_smote, y_smote)

# # undo y_smote refactoring
# y_pred = bst.predict(X_test)
# print(f"ypred before reversing transformation: {y_pred}")
# y_pred = le.inverse_transform(y_pred)
# print(f"ypred after reversing transformation: {y_pred}")

# cm = confusion_matrix(y_test, y_pred)
# rmse = np.sqrt(MSE(y_test, y_pred))

# print(cm)
# print(f"RMSE: {rmse}")

ypred before reversing transformation: [0 3 2 ... 3 3 1]
ypred after reversing transformation: [1 4 3 ... 4 4 2]
[[222  74  14  27]
 [ 80 344  74  84]
 [ 28 162 170 162]
 [ 58 120  97 521]]
RMSE: 1.0353605122153373


In [21]:
# create values for hyperparameter tuning
parameter_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.3],
             'n_estimators': [100, 200, 500, 1000, 2000],
             'colsample_bytree': [0.1, 0.5, 0.7, 1.0]}

In [22]:
clf = xgb.XGBClassifier(random_state = 2001)

cross_val = KFold(5, random_state = 2001, shuffle=True)

clf_grid = GridSearchCV(estimator = clf,
                                param_grid = parameter_grid,
                                scoring = "neg_mean_squared_error",
                                cv = cross_val,
                                verbose = 1,
                                return_train_score = True)

In [23]:
clf_grid.fit(X_smote, y_smote)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=2001, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     i...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=2001, ...),
             param_grid={'colsample_bytree': [0.1, 0.5, 0.7, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1, 0.3],
                         'n_estimators': [100, 200, 500, 1000, 2000]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [24]:
clf_grid.cv_results_
print(clf_grid.best_score_)
print(clf_grid.best_params_)
print(clf_grid.best_index_)
clf_grid.cv_results_['params'][clf_grid.best_index_]

-0.7029368306847521
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'n_estimators': 1000}
33


{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'n_estimators': 1000}

In [25]:
# convert results to pandas df
cv_results_dict = pd.DataFrame.from_dict(clf_grid.cv_results_)

# write to csv for future reference
cv_results_dict.to_csv("/home/jupyter/final_project/xgb_cv_results.csv")

cv_results_dict

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,1.768965,0.035886,0.040885,0.020003,0.1,0.01,100,"{'colsample_bytree': 0.1, 'learning_rate': 0.0...",-1.071877,-0.985860,...,-1.013591,0.034698,80,-0.827735,-0.794245,-0.789039,-0.811254,-0.803201,-0.805095,0.013635
1,3.480168,0.082015,0.058982,0.000561,0.1,0.01,200,"{'colsample_bytree': 0.1, 'learning_rate': 0.0...",-1.023566,-0.981540,...,-1.004322,0.030071,79,-0.776763,-0.739442,-0.746022,-0.747029,-0.749190,-0.751689,0.012952
2,8.484597,0.076963,0.174741,0.041907,0.1,0.01,500,"{'colsample_bytree': 0.1, 'learning_rate': 0.0...",-0.912019,-0.889631,...,-0.909728,0.021197,76,-0.616185,-0.605775,-0.600275,-0.598056,-0.605814,-0.605221,0.006272
3,16.870224,0.259230,0.355146,0.019923,0.1,0.01,1000,"{'colsample_bytree': 0.1, 'learning_rate': 0.0...",-0.846819,-0.835428,...,-0.848368,0.018492,68,-0.424475,-0.403261,-0.421528,-0.411667,-0.418737,-0.415933,0.007628
4,33.750319,1.261094,0.746841,0.024281,0.1,0.01,2000,"{'colsample_bytree': 0.1, 'learning_rate': 0.0...",-0.791830,-0.813040,...,-0.803740,0.016039,58,-0.186309,-0.170595,-0.175800,-0.170677,-0.181970,-0.177070,0.006225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,4.400641,0.063208,0.041248,0.014610,1.0,0.3,100,"{'colsample_bytree': 1.0, 'learning_rate': 0.3...",-0.687353,-0.763158,...,-0.757388,0.036248,36,-0.013455,-0.008839,-0.013062,-0.008838,-0.015516,-0.011942,0.002668
76,8.081089,0.169628,0.062616,0.007492,1.0,0.3,200,"{'colsample_bytree': 1.0, 'learning_rate': 0.3...",-0.684996,-0.727808,...,-0.729261,0.028217,21,-0.000000,-0.000000,-0.000000,-0.000000,-0.000393,-0.000079,0.000157
77,16.640020,0.201813,0.148675,0.001219,1.0,0.3,500,"{'colsample_bytree': 1.0, 'learning_rate': 0.3...",-0.669285,-0.705420,...,-0.720698,0.030012,11,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000
78,27.887004,0.362617,0.318817,0.004580,1.0,0.3,1000,"{'colsample_bytree': 1.0, 'learning_rate': 0.3...",-0.664179,-0.721917,...,-0.719675,0.029154,9,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000


In [ ]:
# graph rmse by each hyperparameter to visualize 

# could even use plotly for a 3D plot

# fit cross_val with best estimator in training data to get various outcomes beyond rmse

# feature importance in training data

In [ ]:
# fit cross_val to test data, get predicted values

# compare predicted values to actual y_test

# feature importance in test data


# EXPERIMENTATION

In [24]:
# create domain for hyperopt hyperparameter tuning
# for more on hyperopt: https://mlwhiz.com/blog/2017/12/28/hyperopt_tuning_ml_model/

space = {'eta': hp.choice('eta', [0.1, 0.01, 0.001]),
         'n_estimators': hp.quniform('n_estimators', 100, 1000, 100),
         'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1)
}

# set up objective function
def objective(space):
    print(space)
    clf = xgb.XGBClassifier(n_estimators = int(space['n_estimators']),
                            eta = space['eta'],
                            colsample_bytree = space['colsample_bytree'])
    best_score = cross_val_score(clf, X_smote, y_smote, scoring='neg_mean_squared_error', cv=5).mean()
    return (-1*best_score) # multiply by negative 1 because fmin minimizes so we don't want negative RMSE

# run algorithm
best = fmin(fn = objective,
            space = space,
            max_evals = 20,
            rstate = np.random.default_rng(2001),
            algo = tpe.suggest)
print(best)

{'colsample_bytree': 0.6379571229213834, 'eta': 0.01, 'n_estimators': 500.0}
{'colsample_bytree': 0.2756466829735948, 'eta': 0.1, 'n_estimators': 100.0}     
{'colsample_bytree': 0.3046744246673796, 'eta': 0.001, 'n_estimators': 100.0}   
{'colsample_bytree': 0.3243534369106372, 'eta': 0.001, 'n_estimators': 700.0}   
{'colsample_bytree': 0.10849623877720022, 'eta': 0.1, 'n_estimators': 200.0}    
{'colsample_bytree': 0.7098828568886123, 'eta': 0.1, 'n_estimators': 1000.0}    
{'colsample_bytree': 0.24486825937282405, 'eta': 0.01, 'n_estimators': 200.0}   
{'colsample_bytree': 0.9303623836206345, 'eta': 0.01, 'n_estimators': 200.0}   
{'colsample_bytree': 0.25039070696710786, 'eta': 0.01, 'n_estimators': 600.0}  
{'colsample_bytree': 0.5467074536134859, 'eta': 0.01, 'n_estimators': 100.0}   
{'colsample_bytree': 0.9962736188722175, 'eta': 0.01, 'n_estimators': 700.0}    
{'colsample_bytree': 0.6292854555168056, 'eta': 0.01, 'n_estimators': 700.0}    
{'colsample_bytree': 0.368805384993

In [30]:
# instantiate the regressor
xgboost_class = xgb.XGBClassifier(random_state = 2001)

# create random search object
# neg_mean_squared_error per https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
random_xgb = RandomizedSearchCV(
    estimator = xgboost_class,
    param_distributions = parameter_grid,
    scoring = 'neg_mean_squared_error',
    n_iter = 10,
    cv = 5,
    refit = True,
    return_train_score = True,
    verbose = 1)

# fit grid search object to imputed and oversampled training data
random_xgb.fit(X_smote, y_smote)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=2001, ...),
                   param_distributions={'colsample_bytree': [0.1, 0.5, 0.7,
                                                             1.0],
                                        'learning_rate': [0.1, 0.01, 0.001],
                                        'n_estimators': [50, 100, 250, 500]},
                   return_train_score=True, scoring='neg_mean_squared_error',
                   verbose=1)

In [32]:
# print values used for search
print(f"\n The learning rate parameters searched:\n {random_xgb.cv_results_['param_learning_rate']}")
print(f"\n The number of estimators parameters searched:\n {random_xgb.cv_results_['param_n_estimators']}")
print(f"\n The number of features considered at each split, parameters searched:\n {random_xgb.cv_results_['param_colsample_bytree']}")

print(random_xgb.cv_results_)

print("\n The best score across ALL searched params:\n", random_xgb.best_score_)
print("\n The best parameters across ALL searched params:\n", random_xgb.best_params_)


 The learning rate parameters searched:
 [0.01 0.1 0.001 0.001 0.001 0.01 0.1 0.1 0.001 0.1]

 The number of estimators parameters searched:
 [100 100 250 100 250 250 100 100 50 50]

 The number of features considered at each split, parameters searched:
 [0.1 1.0 0.5 1.0 0.1 0.1 0.1 0.5 0.5 0.1]
{'mean_fit_time': array([1.77714324, 4.76551232, 8.17387962, 5.07713327, 4.22043757,
       4.24490185, 1.79613562, 3.12225575, 1.68987536, 0.88884268]), 'std_fit_time': array([0.05652633, 0.28847723, 0.46372596, 0.36441579, 0.19229218,
       0.18786928, 0.11770793, 0.20453196, 0.09721584, 0.051469  ]), 'mean_score_time': array([0.03293095, 0.03158164, 0.08445587, 0.03088861, 0.08526173,
       0.0760529 , 0.0313159 , 0.03127069, 0.01725502, 0.01718459]), 'std_score_time': array([0.00146008, 0.00051271, 0.02035005, 0.00019459, 0.02172211,
       0.0028081 , 0.00023523, 0.00032256, 0.00025947, 0.0002154 ]), 'param_n_estimators': masked_array(data=[100, 100, 250, 100, 250, 250, 100, 100, 50, 50